In [1]:
# --------IMPORTS---------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
from datetime import datetime, timedelta
from sqlalchemy import create_engine
from sqlalchemy import Table, MetaData

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, RandomizedSearchCV

In [40]:
# --- MAIN EXECUTION ---
query ="""SELECT 
        p.policy_no,
        p.inception_date_new,
        p.expiry_date,
        p.split,
        p.sum_insured,
        p.branch,
        p.cust_type,
        p.cust_post_code,
        p.cust_gender,
        p.cust_age,
        p.vehicle_age,
        p.make,
        p.model,
        p.cc,
        p.cust_state,
        p.xc_veh_renewal_no,
        p.ncb_perc,
        p.pref_segment,
        0::numeric AS GWP,
        SUM(p.gic) AS GIC,
        SUM(p.noc_total) AS noc_total
    FROM ads.fct_mot_clm p
    LEFT JOIN ADS.MST_CONTRACT_TYPE c1
        ON p.CONTRACT_TYPE = c1.CONTRACT_TYPE_CODE
    INNER JOIN (
        SELECT * 
        FROM ADS.CTL_CALENDAR 
        WHERE CALENDAR_DT = (
            SELECT MAX(CALENDAR_DT) 
            FROM ADS.CTL_CALENDAR 
            WHERE STATUS = 'SUCCESS'
        )
    ) c ON p.uw_date >= DATE '2025-05-01'
    WHERE 
        c1.CONTRACT_TYPE_LONG_DESC ILIKE '%PRIVATE CAR%' AND 
        p.cover = 'CO'
    GROUP BY 
        p.policy_no,
        p.inception_date_new,
        p.expiry_date,
        p.split,
        p.sum_insured,
        p.branch,
        p.cust_type,
        p.cust_post_code,
        p.cust_gender,
        p.cust_age,
        p.vehicle_age,
        p.make,
        p.model,
        p.cc,
        p.cust_state,
        p.xc_veh_renewal_no,
        p.ncb_perc,
        p.pref_segment
    
    UNION ALL

    SELECT 
        p.policy_no,
        p.inception_date_final,
        p.expiry_date,
        p.split,
        p.sum_insured,
        p.branch,
        p.cust_type,
        p.cust_post_code,
        p.cust_gender,
        p.cust_age,
        p.vehicle_age,
        p.make,
        p.model,
        p.cc,
        p.cust_state,
        p.xc_veh_renewal_no,
        p.ncb_perc,
        p.pref_segment,
        SUM(p.gwp) AS GWP,
        0::numeric AS GIC,
        0::numeric AS noc_total
    FROM ads.fct_mot_pol p
    LEFT JOIN ADS.MST_CONTRACT_TYPE c1
        ON p.CONTRACT_TYPE = c1.CONTRACT_TYPE_CODE
    INNER JOIN (
        SELECT * 
        FROM ADS.CTL_CALENDAR 
        WHERE CALENDAR_DT = (
            SELECT MAX(CALENDAR_DT) 
            FROM ADS.CTL_CALENDAR 
            WHERE STATUS = 'SUCCESS'
        )
    ) c ON p.uw_date >= DATE '2025-05-01'
    WHERE 
        c1.CONTRACT_TYPE_LONG_DESC ILIKE '%PRIVATE CAR%' AND 
        p.cover = 'CO'
    GROUP BY 
        p.policy_no,
        p.inception_date_final,
        p.expiry_date,
        p.split,
        p.sum_insured,
        p.branch,
        p.cust_type,
        p.cust_post_code,
        p.cust_gender,
        p.cust_age,
        p.vehicle_age,
        p.make,
        p.model,
        p.cc,
        p.cust_state,
        p.xc_veh_renewal_no,
        p.ncb_perc,
        p.pref_segment;  
        ;"""
# --- DB CONNECTION ---
try:
    connection = psycopg2.connect(
        host = "172.16.10.212",
        dbname = "PIB_PRD",
        user = "usr_app_pdi",
        password = 'Usr@ppPd!#123',
        port = "5432"
    )
    print("Connected to the database")

    cursor = connection.cursor()
    dff = pd.read_sql_query(query, connection)
    print("DataFrame Fetched Successfully!")
    # print(dff.head())

except Exception as e:
    print("Error: Unable to connect to the database")
    print(e)

# finally:
#     # Close the cursor and connection
#     if cursor:
#         cursor.close()
#     if connection:
#         connection.close()

Connected to the database


C:\Users\ananya.datta\AppData\Local\Temp\ipykernel_18596\2782923281.py:130: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dff = pd.read_sql_query(query, connection)


DataFrame Fetched Successfully!


In [41]:
# ------Trial purposes-------
dff.to_csv('POLNCLAIM_27-06.csv',index = False)
dff = pd.read_csv(r'C:\Users\ananya.datta\OneDrive - Fairfax Asia Ltd\Desktop\Project 1 - Predictive Premium Pricing Modelling\Notebooks.py\POLNCLAIM_27-06.csv')
# dff = pd.read_csv(r'C:\Users\ananya.datta\OneDrive - Fairfax Asia Ltd\Desktop\Project 1 - Predictive Premium Pricing Modelling\Notebooks.py\POLNCLAIM_11-06.csv')
# dff.shape

In [42]:
dff.shape
# dff.inception_date_new.value_counts().sort_index()

(26690, 21)

In [43]:
df = dff.copy()
# df = df[df['inception_date_new'] > '2025-04-30']
df = df.groupby(['policy_no','inception_date_new','expiry_date','split']).agg({
    'gwp': 'sum',          
    'sum_insured': 'sum',  
    'gic': 'sum',          
    'noc_total': 'sum',    
    'branch': 'first',     
    'cust_type': 'first',  
    'cust_post_code': 'first',  
    'cust_state': 'first', 
    'cust_gender': 'first', 
    'cust_age': 'first',   
    'vehicle_age': 'first', 
    'make': 'first',      
    'model': 'first',      
    'cc': 'first',         
    'xc_veh_renewal_no': 'first', 
    'ncb_perc': 'first',  
    'pref_segment': 'first' }).reset_index()

In [44]:
def categorize_cc(f):
    # Define the bins and labels
    bins = [0, 1000, 1500, 2000, 3000, 4000, np.inf]
    labels = ['0-1000', '1000-1500', '1500-2000', '2000-3000', '3000-4000', '4000+']
    
    # Apply the categorization to the 'cc' column using pd.cut
    f['cc'] = pd.cut(f['cc'], bins=bins, labels=labels, right=False)
    
    return f
    
def process_cust_age(f, lowerb=20, upperb=80):
    # Cap and floor bounds for 'cust_age'
    f['cust_age'] = f['cust_age'].apply(lambda x: max(min(x, upperb), lowerb))
    
    # Fill the missing values with the sampled values
    missing_indices = f[f['cust_age'].isnull()].index
    non_missing_values = f['cust_age'].dropna()  # Get non-null values
    random_samples = np.random.choice(non_missing_values, size=len(missing_indices), replace=True)
    f.loc[missing_indices, 'cust_age'] = random_samples
    return f

def process_cust_gender(f):
     # Calculate the proportions of 'M' and 'F' in the entire dataset
    f['cust_gender'] = f['cust_gender'].replace([''], np.nan)
    gender_counts = f['cust_gender'].value_counts(normalize=True)
    prob_m = gender_counts.get('M', 0)  # Proportion of 'M'
    prob_f = gender_counts.get('F', 0)  # Proportion of 'F'
    # Make sure the probabilities sum to 1
    total_prob = prob_m + prob_f
    if total_prob != 1:
        prob_m /= total_prob
        prob_f /= total_prob
    # Define the imputation function
    def fill_gender(row):
        if pd.isna(row['cust_gender']):
            # Sample a gender based on the overall proportions
            return np.random.choice(['M', 'F'], p=[prob_m, prob_f])
        return row['cust_gender']
    f['cust_gender'] = f.apply(fill_gender, axis=1)
    return f

def process_model(f, top_n=10):
    # Get the top N most frequent models
    model_distribution = f['model'].dropna().value_counts()
    top_n_models = model_distribution.head(top_n)
    model_proportions = top_n_models / top_n_models.sum()
    # Generate a list of model values to fill in the missing 'model' entries, proportionally
    missing_indices = f[f['model'].isnull()].index
    # Sample from the top N models using the calculated proportions
    fill_values = np.random.choice(top_n_models.index, size=len(missing_indices), p=model_proportions)
    # Fill the missing values in the 'model' column
    f.loc[missing_indices, 'model'] = fill_values

    return f

def process_make(f, top_n=10):
    # Get the top N most frequent models
    make_distribution = f['make'].dropna().value_counts()
    top_n_make = make_distribution.head(top_n)
    make_proportions = top_n_make / top_n_make.sum()
    # Generate a list of model values to fill in the missing 'model' entries, proportionally
    missing_indices = f[f['make'].isnull()].index
    # Sample from the top N models using the calculated proportions
    fill_values = np.random.choice(top_n_make.index, size=len(missing_indices), p=make_proportions)
    # Fill the missing values in the 'model' column
    f.loc[missing_indices, 'make'] = fill_values

    return f

def process_postcode(f, top_n=10):
    # Get the top N most frequent models
    postcode_distribution = f['cust_post_code'].dropna().value_counts()
    top_n_postcode = postcode_distribution.head(top_n)
    postcode_proportions = top_n_postcode / top_n_postcode.sum()
    # Generate a list of model values to fill in the missing 'model' entries, proportionally
    missing_indices = f[f['cust_post_code'].isnull()].index
    # Sample from the top N models using the calculated proportions
    fill_values = np.random.choice(top_n_postcode.index, size=len(missing_indices), p=postcode_proportions)
    # Fill the missing values in the 'model' column
    f.loc[missing_indices, 'cust_post_code'] = fill_values

    return f

def process_lossratio(f):
    # Calculate the 'loss_ratio' column
    f['loss_ratio'] = f.apply(lambda row: 'not there' if row['gic'] == 0 and row['gwp'] == 0 
                              else 0 if row['gic'] == 0 
                              else 'not defined' if row['gwp'] == 0 
                              else row['gic'] / row['gwp'], axis=1)
    
    # Remove rows where 'loss_ratio' is 'not there' or 'not defined'
    f = f[~f['loss_ratio'].isin(['not there', 'not defined'])]
    return f

In [45]:
df = process_cust_age(df)
df = process_cust_gender(df)
df = process_model(df)
df = process_make(df)
df = process_postcode(df)
df = process_lossratio(df)
df = categorize_cc(df)

In [46]:
def initial_preprocess(df):
    if not np.issubdtype(df['inception_date_new'].dtype, np.datetime64):
        df['inception_date_new'] = pd.to_datetime(df['inception_date_new'])
    df['inception_year'] = df['inception_date_new'].dt.year
    df['loss_ratio'] = df['loss_ratio'].astype('int64')
    df = df[df['branch'] != 'H5']
    return df

def extract_date_parts(df):
    if not np.issubdtype(df['inception_date_new'].dtype, np.datetime64):
        df['inception_date_new'] = pd.to_datetime(df['inception_date_new'])
    df['inception_month'] = df['inception_date_new'].dt.month
    df['inception_year'] = df['inception_date_new'].dt.year
    df = df.sort_values('inception_date_new')
    return df

def preserve_original(df, cols):
    return df[cols].copy()

def drop_unnecessary_columns(df, cols_to_drop):
    return df.drop(cols_to_drop, axis=1)

def convert_dtypes_for_catboost(df, convert_float_to_int=True):
    df["inception_year"] = df["inception_year"].astype('int64')
    df["inception_month"] = df["inception_month"].astype('int64')
    df["cc"] = df["cc"].astype('object')
    return df 

def prepare_data(df, preserve_cols, drop_cols, convert_float_to_int=True):
    preserved_df = preserve_original(df, preserve_cols)
    df = drop_unnecessary_columns(df, drop_cols)
    df = convert_dtypes_for_catboost(df, convert_float_to_int=convert_float_to_int)
    return df, preserved_df

In [47]:
preserve_cols = ['policy_no', 'inception_date_new', 'expiry_date', 'noc_total']
drop_cols = ['policy_no', 'inception_date_new', 'expiry_date', 'noc_total']

# testing dataframe
tst1 = initial_preprocess(df)
tst1 = extract_date_parts(df)
tst1, tst1_og = prepare_data(tst1, preserve_cols, drop_cols)

In [48]:
def categorize_columns_for_catboost(df):
    num = []
    cat = []
    for column in df.columns:
        if df[column].dtype in ['int64', 'float64']:  # For numerical columns
            num.append(column)
            if df[column].dtype == 'float64':  # Convert float to int
                df[column] = df[column].astype(np.int64)
        elif df[column].dtype == 'object':  # For categorical columns (string type)
            cat.append(column)
    
    return num, cat

numerical,categorical = categorize_columns_for_catboost(tst1)
numerical.pop(0)

'gwp'

In [49]:
best_model

In [50]:
# Create an instance of the model
best_model = CatBoostRegressor()
best_model.load_model('catboostRegressorfinal')

In [51]:
dff.dtypes

policy_no              object
inception_date_new     object
expiry_date            object
split                  object
sum_insured           float64
branch                 object
cust_type              object
cust_post_code          int64
cust_gender            object
cust_age              float64
vehicle_age           float64
make                   object
model                  object
cc                    float64
cust_state             object
xc_veh_renewal_no       int64
ncb_perc              float64
pref_segment            int64
gwp                   float64
gic                   float64
noc_total             float64
dtype: object

In [52]:
def predict_and_merge(model, df, df_og, numerical_columns, categorical_columns, target_col='gwp'):
    # Prepare the features (X) and target (y)
    X = df[numerical_columns + categorical_columns]
    y = df[target_col]

    # Make predictions (without passing 'cat_features')
    y_pred = model.predict(X)

    # Assign the predicted values to the dataframe
    df = df.assign(predicted_gwp=y_pred)

    # Convert 'inception_date_new' to object type in the original dataframe
    df_og['inception_date_new'] = df_og['inception_date_new'].astype('object')

    # Merge the original dataframe with the predictions
    df = df_og.merge(df, left_index=True, right_index=True, how='left')

    # Convert 'inception_date_new' back to datetime format
    df['inception_date_new'] = pd.to_datetime(df['inception_date_new']).dt.date
    
    return df

# Apply the function to tst1
tst1 = predict_and_merge(best_model, tst1, tst1_og, numerical, categorical)

In [30]:
# print(tst1[categorical].dtypes)
# tst1.split.unique()

In [31]:
# for col in categorical:
#     print(f"Unique values in {col}: {tst1[col].unique()}")

In [32]:
# # Check the dataframe for rows containing 'NONACT'
# problematic_rows = tst1[tst1.apply(lambda row: row.astype(str).str.contains('NONACT').any(), axis=1)]
# problematic_rows

In [54]:
tst1.columns
tst1.head()
# tst1.shape

,policy_no,inception_date_new,expiry_date,noc_total,split,gwp,sum_insured,gic,branch,cust_type,...,make,model,cc,xc_veh_renewal_no,ncb_perc,pref_segment,loss_ratio,inception_year,inception_month,predicted_gwp
11705,V7205347,2025-05-01,2026-04-30,0.0,ACT,329,20700,0,T1,P,...,ISUZU,D MAX,2000-3000,7,55,0,0,2025,5,212.875480
5015,V6765758,2025-05-01,2026-04-30,0.0,NONACT,472,12900,0,W8,P,...,TOYOTA,VIOS,1000-1500,2,55,0,0,2025,5,290.455670
5014,V6765758,2025-05-01,2026-04-30,0.0,ACT,187,12900,0,W8,P,...,TOYOTA,VIOS,1000-1500,2,55,0,0,2025,5,151.302975
4963,V6762151,2025-05-01,2026-04-30,0.0,NONACT,373,18100,0,J1,P,...,TOYOTA,CAMRY,1500-2000,4,55,3,0,2025,5,310.951137
4962,V6762151,2025-05-01,2026-04-30,0.0,ACT,316,18100,0,J1,P,...,TOYOTA,CAMRY,1500-2000,4,55,3,0,2025,5,191.217097


In [ ]:
# --- DATABASE CONNECTION AND UPLOAD ---
# Define the database connection parameters
host = "172.16.10.212"
dbname = "PIB_PRD"
user = "usr_app_pdi"
password = 'Usr@ppPd!#123'
port = "5432"  # Default is 5432

# Safely encode credentials
safe_user = quote_plus(user)
safe_password = quote_plus(password)

# Build connection string
connection_string = f'postgresql+psycopg2://{safe_user}:{safe_password}@{host}:{port}/{dbname}'
engine = create_engine(connection_string)
tst1.to_sql('pol_clm_gwp_prediction_May25', engine, schema='ads', if_exists='replace', index=False)
print("Dataframe uploaded successfully.")

In [ ]:
if cursor:
    cursor.close()
if connection:
    connection.close()